## Ссылки на статьи
- https://habr.com/ru/post/483302/
- https://habr.com/ru/post/305378/
- http://datalytics.ru/all/rabotaem-s-api-google-drive-s-pomoschyu-python/
- https://docs.gspread.org/en/latest/user-guide.html#opening-a-spreadsheet

In [1]:
# Подключаем библиотеки
import httplib2 
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials

from __future__ import print_function
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
import pprint
import io

pp = pprint.PrettyPrinter(indent=4)

In [2]:
import pandas as pd

In [3]:
# библиотека работы с Гугл шитс
# https://docs.gspread.org/en/latest/user-guide.html#opening-a-spreadsheet
import gspread
import gspread_dataframe as gd


## файл с ключом доступа

In [4]:
# Ссылка на файл с закрытым ключом
CREDENTIALS_FILE = r'C:\Users\ezhmui04\Documents\Ключ доступа к GoogleCloud\directdashboard-734cbb03b84d.json' 
# создание SCOPE только для драйва
SCOPES_drive = ['https://www.googleapis.com/auth/drive']

In [5]:
credentials_drive = service_account.Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=SCOPES_drive)
gc = gspread.service_account(CREDENTIALS_FILE)


## создание пустого файла

In [6]:
# Читаем ключи из файла
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])

httpAuth = credentials.authorize(httplib2.Http()) # Авторизуемся в системе
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth) # Выбираем работу с таблицами и 4 версию API

spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'Первый тестовый документ', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': 'Лист номер один',
                               'gridProperties': {'rowCount': 100, 'columnCount': 15}}}]
}).execute()
spreadsheetId = spreadsheet['spreadsheetId'] # сохраняем идентификатор файла
print('https://docs.google.com/spreadsheets/d/' + spreadsheetId)

https://docs.google.com/spreadsheets/d/1vC_HAzqBwV1DiVg24rseHj8TeTJF_YVeQO4BGTec27g


## разрешение на редактирование файла

In [7]:
# выдача разрешения на редактирование файла 
driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth) # Выбираем работу с Google Drive и 3 версию API
access = driveService.permissions().create(
    fileId = spreadsheetId,
    body = {'type': 'user', 'role': 'reader', 'emailAddress': 'rostov78@mail.ru'},  # Открываем доступ на редактирование
    fields = 'id'
).execute()

# role can be also 'writer'

## заполнение файла

In [8]:
results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetId, body = {
    "valueInputOption": "USER_ENTERED", # Данные воспринимаются, как вводимые пользователем (считается значение формул)
    "data": [
        {"range": "Лист номер один!B2:D5",
         "majorDimension": "ROWS",     # Сначала заполнять строки, затем столбцы
         "values": [
                    ["Ячейка B2", "Ячейка C2", "Ячейка D2"], # Заполняем первую строку
                    ['25', "=6*6", "=sin(3,14/2)"]  # Заполняем вторую строку
                   ]}
    ]
}).execute()

## чтение данных из файла

In [9]:
ranges = ["Лист номер один!A2:F8"] # 
          
results = service.spreadsheets().values().batchGet(spreadsheetId = spreadsheetId, 
                                     ranges = ranges, 
                                     valueRenderOption = 'FORMATTED_VALUE',  
                                     dateTimeRenderOption = 'FORMATTED_STRING').execute() 
sheet_values = results['valueRanges'][0]['values']
print(sheet_values)

[['', 'Ячейка B2', 'Ячейка C2', 'Ячейка D2'], ['', '25', '36', '0,9999996829']]


## перемещение файла из корневой папки

In [11]:
driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth) # Выбираем работу с Google Drive и 3 версию API


In [12]:
# создание папки куда переместим файл
file_metadata = {
            'name': 'Direct_data',
            'mimeType': 'application/vnd.google-apps.folder'
        }

# pylint: disable=maybe-no-member
file = driveService.files().create(body=file_metadata, fields='id'
                                      ).execute()
print(F'Folder ID: "{file.get("id")}".')

folder = file.get("id")

Folder ID: "1E4NxuaGuvGkW05UUC1UdKNqFQIwI9iD_".


In [13]:
# выдача разрешения на редактирование папки (но я ее все равно не вижу) 
access = driveService.permissions().create(
    fileId = folder,
    body = {'type': 'user', 'role': 'reader', 'emailAddress': 'rostov78@mail.ru'},  # Открываем доступ на редактирование
    fields = 'id'
).execute()

In [14]:
# выбираем работу с драйовом
service = build('drive', 'v3', credentials=credentials_drive)

file = service.files().get(fileId=spreadsheetId, fields='parents').execute()
previous_parents = ",".join(file.get('parents'))
# Move the file to the new folder
file = service.files().update(fileId=spreadsheetId, addParents=folder, removeParents=previous_parents, fields='id, parents').execute()

## дополнительная библиотека работы с драйвом

In [15]:
# библиотека работы с Гугл шитс
#import gspread

#gc = gspread.service_account(CREDENTIALS_FILE)

# Open a spreadsheet by title
#sh = gc.open("Iris Data")

# Get the first sheet
#wk = sh.sheet1

# Update a range of cells using the top left corner address
#wk.update('A1', [['Species', 'Sepal length'], ['Iris setosa', 5.1]])

# Format the header
#wk.format('A1:B1', {'textFormat': {'bold': True}})

# Open a sheet from a spreadsheet in one go
#sht2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1KFywsz-4a5blbYpv9_mviMrPsYvvXSJh49rkx_Q0VHA/edit#gid=0')

# очистка всего листа
# Get the first sheet
#wk = sht2.sheet1
#wk.clear()

# создание файла
#sh = gc.create('A new spreadsheet')

# изменение прав доступа к файлу
#sh.share('rostov78@mail.ru', perm_type='user', role='reader')


## создание тестовых датафреймов

In [16]:
df = pd.util.testing.makeDataFrame()

df.head()

C:\Users\ezhmui04\PycharmProjects\pythonProject\venv\lib\site-packages\pandas\util\__init__.py:16: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing


,A,B,C,D
UFxCofXepO,-0.470408,0.430277,-0.043902,-0.408101
gFURJxB0fS,-1.076257,1.099781,0.920815,2.779751
nXSAI00yF8,-0.105437,1.296588,2.614192,-0.337523
GzuoE9DNOP,-0.809165,0.354133,1.053262,1.655058
O7JGQ1Xo4O,1.010462,-1.257395,-1.847029,0.778408


In [17]:
df_2 = pd.util.testing.makeDataFrame()

df_2.head()

,A,B,C,D
smCwRl4tMK,-0.510449,-0.916556,-0.524588,-1.203840
5JWUyFQ8fl,2.357641,0.705645,0.424248,0.744603
AqR9XUk4fT,0.163010,1.511941,0.025359,0.623509
ICRW9EFX81,-0.281312,-0.309317,0.732052,-1.358567
aeerFZGrJE,2.085723,-0.321015,1.982856,0.646546


In [18]:
#Open a sheet from a spreadsheet in one go
sht2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1KFywsz-4a5blbYpv9_mviMrPsYvvXSJh49rkx_Q0VHA/edit#gid=0')

# очистка всего листа
# Get the first sheet
wk = sht2.sheet1
wk.clear()

{'spreadsheetId': '1KFywsz-4a5blbYpv9_mviMrPsYvvXSJh49rkx_Q0VHA',
 'clearedRange': "'Лист номер один'!A1:O100"}

In [19]:
# запись в файл датафрейма целиком
wk.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1KFywsz-4a5blbYpv9_mviMrPsYvvXSJh49rkx_Q0VHA',
 'updatedRange': "'Лист номер один'!A1:D31",
 'updatedRows': 31,
 'updatedColumns': 4,
 'updatedCells': 124}

In [20]:
# добавление в файл нового датафрейма
gsc = gspread.authorize(credentials)
sheet = sht2
params = {'valueInputOption': 'USER_ENTERED'}
body = {'values': df_2.values.tolist()}
sheet.values_append(f'wkA2:D2', params, body)

{'spreadsheetId': '1KFywsz-4a5blbYpv9_mviMrPsYvvXSJh49rkx_Q0VHA',
 'tableRange': "'Лист номер один'!A1:D31",
 'updates': {'spreadsheetId': '1KFywsz-4a5blbYpv9_mviMrPsYvvXSJh49rkx_Q0VHA',
  'updatedRange': "'Лист номер один'!A32:D61",
  'updatedRows': 30,
  'updatedColumns': 4,
  'updatedCells': 120}}